# Trucation of the Pauli Weight during propagation

In [4]:
import pennylane as qml # To define the circuit
from pprop.propagator import Propagator # For Pauli Propagation

In [15]:
# Definition of a very large circuit
num_qubits = 64
def ansatz(params):
    for q in range(num_qubits):
        qml.Hadamard(wires=q)
        qml.RY(params[q], wires=q)
        
    for q_control, q_target in zip(range(0, num_qubits, 2), range(1, num_qubits, 2)):
        qml.CNOT(wires=[q_control, q_target])

    qml.Barrier()

    for q in range(num_qubits):
        qml.RY(params[num_qubits+q], wires=q)

    for q_control, q_target in zip(range(1, num_qubits, 2), range(2, num_qubits, 2)):
        qml.CNOT(wires=[q_control, q_target])

    qml.Barrier()

    for q in range(num_qubits):
        qml.RY(params[2*num_qubits+q], wires=q)


    return [qml.expval(qml.PauliZ(qubit)) for qubit in range(num_qubits)]


In [18]:
prop = Propagator(ansatz, k1=4, k2=None)
prop

Propagator
  Number of qubits : 64
  Trainable parameters : 192
  Cutoff 1: 4 | Cutoff 2: None
  Observables [Z(0), Z(1), Z(2), Z(3), Z(4), Z(5), Z(6), Z(7), Z(8), Z(9), Z(10), Z(11), Z(12), Z(13), Z(14), Z(15), Z(16), Z(17), Z(18), Z(19), Z(20), Z(21), Z(22), Z(23), Z(24), Z(25), Z(26), Z(27), Z(28), Z(29), Z(30), Z(31), Z(32), Z(33), Z(34), Z(35), Z(36), Z(37), Z(38), Z(39), Z(40), Z(41), Z(42), Z(43), Z(44), Z(45), Z(46), Z(47), Z(48), Z(49), Z(50), Z(51), Z(52), Z(53), Z(54), Z(55), Z(56), Z(57), Z(58), Z(59), Z(60), Z(61), Z(62), Z(63)]
 0: ──H──RY─╭●──||──RY─────||──RY─┤  <Z>
 1: ──H──RY─╰X──||──RY─╭●──||──RY─┤  <Z>
 2: ──H──RY─╭●──||──RY─╰X──||──RY─┤  <Z>
 3: ──H──RY─╰X──||──RY─╭●──||──RY─┤  <Z>
 4: ──H──RY─╭●──||──RY─╰X──||──RY─┤  <Z>
 5: ──H──RY─╰X──||──RY─╭●──||──RY─┤  <Z>
 6: ──H──RY─╭●──||──RY─╰X──||──RY─┤  <Z>
 7: ──H──RY─╰X──||──RY─╭●──||──RY─┤  <Z>
 8: ──H──RY─╭●──||──RY─╰X──||──RY─┤  <Z>
 9: ──H──RY─╰X──||──RY─╭●──||──RY─┤  <Z>
10: ──H──RY─╭●──||──RY─╰X──||──RY─┤  <Z>
1

In [19]:
prop.propagate()

Propagating Z(63): 100%|██████████| 64/64 [00:00<00:00, 149.15it/s]


In [21]:
func = prop.lambdify()

In [22]:
func(qml.numpy.random.rand(prop.num_params))

[tensor(-0.80662229, requires_grad=True),
 tensor(-0.48501462, requires_grad=True),
 tensor(-0.01866235, requires_grad=True),
 tensor(-0.7706757, requires_grad=True),
 tensor(0.14474812, requires_grad=True),
 tensor(-0.2646237, requires_grad=True),
 tensor(-0.30377408, requires_grad=True),
 tensor(0.33392076, requires_grad=True),
 tensor(-0.04230253, requires_grad=True),
 tensor(-0.91800672, requires_grad=True),
 tensor(-0.06971689, requires_grad=True),
 tensor(-0.68583777, requires_grad=True),
 tensor(0.48417193, requires_grad=True),
 tensor(0.07484166, requires_grad=True),
 tensor(0.27805619, requires_grad=True),
 tensor(-0.86474061, requires_grad=True),
 tensor(-0.40908855, requires_grad=True),
 tensor(0.08867266, requires_grad=True),
 tensor(-0.09148497, requires_grad=True),
 tensor(0.02355589, requires_grad=True),
 tensor(-0.05123843, requires_grad=True),
 tensor(-0.54034324, requires_grad=True),
 tensor(-0.23643505, requires_grad=True),
 tensor(-0.50693332, requires_grad=True),
 